In [670]:
# Importing libraries
import numpy as np
import pandas as pd
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,recall_score, f1_score
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, SelectPercentile, RFE, SelectFromModel,chi2
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


%matplotlib inline

In [671]:
# Reading the train.csv by removing the
# last column since it's an empty column
training_data = pd.read_csv("data/symptoms_Data_Training.csv")

In [ ]:
training_data.head(1)

In [ ]:
training_data.drop(training_data.columns[-1], axis=1,inplace=True)
training_data.head(1)

In [674]:
def data_processing(data):
    # List of target labels
    target_labels = ["Diabetes", "Hypertension"]

    # 1. Change all rows not in target labels to "not ill"
    data["prognosis"] = data["prognosis"].apply(
        lambda x: x if x in target_labels else "not ill"
    )

    # Separate majority and minority classes
    majority = data[data["prognosis"] == "not ill"]
    minority_Hypertension = data[data["prognosis"] == "Hypertension"]
    minority_Diabetes = data[data["prognosis"] == "Diabetes"]

    # Upsample minority class
    minority_upsampled_Diabetes = resample(
        minority_Diabetes, replace=True, n_samples=len(majority), random_state=42
    )
    minority_upsampled_Hypertension = resample(
        minority_Hypertension, replace=True, n_samples=len(majority), random_state=42
    )

    # Combine with majority class
    data = pd.concat(
        [majority, minority_upsampled_Diabetes, minority_upsampled_Hypertension]
    )

    print(data["prognosis"].value_counts())

    return data

In [ ]:
data = data_processing(training_data)

In [ ]:
# Data types and non-null counts
print(data.info())

In [ ]:
# Summary for binary columns
data.describe()

In [ ]:
# Check for missing values in each column
data.isnull().sum()

In [ ]:
for col in data.columns[:-1]:  # Exclude the last object column
    print(f"{col}:\n{data[col].value_counts()}\n")

In [ ]:
print(data.columns.tolist())

In [681]:
# correlation_matrix = data.iloc[:, :-1].corr()
# # Create a heatmap of the correlation matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Correlation Matrix for Iris Dataset')
# plt.show()

In [682]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
# plt.title("Correlation Matrix of Binary Features")
# plt.show()


In [683]:
# # Bar plots for each binary feature grouped by the Category
# for col in data.columns[:-1]:
#     sns.barplot(x='prognosis', y=col, data=data)
#     plt.title(f"Distribution of {col} by prognosis")
#     plt.show()


In [ ]:
# Checking whether the dataset is balanced or not
disease_counts = data["prognosis"].value_counts()
temp_df = pd.DataFrame({
    "Disease": disease_counts.index,
    "Counts": disease_counts.values
})

plt.figure(figsize = (18,8))
sns.barplot(x = "Disease", y = "Counts", data = temp_df)
plt.xticks(rotation=90)
plt.show()


In [685]:
# Encoding the target value into numerical
# value using LabelEncoder
encoder = LabelEncoder()
data["prognosis"] = encoder.fit_transform(data["prognosis"])


In [686]:
XX = data.iloc[:,:-1]
y = data.iloc[:, -1]

In [ ]:
# sama

# Assume X is your feature matrix and y is your target variable
selector = SelectKBest(score_func=chi2, k=10)  # Select top 10 features
X = selector.fit_transform(XX, y)

# Get scores and feature indices
scores = selector.scores_
selected_features = selector.get_support(indices=True)
print("Selected feature indices:", selected_features)
# print("Feature scores:", scores)
selected_columns = XX.columns[selector.get_support()].tolist()
print("Selected columns:", selected_columns)


In [ ]:
X_train, X_test, y_train, y_test =train_test_split(
  X, y, test_size = 0.2, random_state = 42)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")


In [ ]:
# Defining scoring metric for k-fold cross validation
def cv_scoring(estimator, X, y):
    return accuracy_score(y, estimator.predict(X))

# Initializing Models
models = {
    "SVC":SVC(),
    "Gaussian NB":GaussianNB(),
    "Random Forest":RandomForestClassifier(random_state=18),
    "Decision Tree":DecisionTreeClassifier(criterion="gini")
}

# Producing cross validation score for the models
for model_name in models:
    model = models[model_name]
    scores = cross_val_score(model, X, y, cv = 10,
                             n_jobs = -1,
                             scoring = cv_scoring)
    print("=="*30)
    print(model_name)
    print(f"Scores: {scores}")
    print(f"Mean Score: {np.mean(scores)}")


In [ ]:
# Training and testing SVM Classifier
svm_model = SVC()
svm_model.fit(X_train, y_train)
preds = svm_model.predict(X_test)

print(
    f"Accuracy on train data by SVM Classifier: {accuracy_score(y_train, svm_model.predict(X_train))*100}\
     precision on train data by SVM Classifier: {precision_score(y_train, svm_model.predict(X_train),average="weighted")*100}\
     recall score on train data by SVM Classifier: {recall_score(y_train, svm_model.predict(X_train),average="weighted")*100} \
     f1 score on train data by SVM Classifier: {f1_score(y_train, svm_model.predict(X_train),average="weighted")*100}"
)

print(
    f"Accuracy on test data by SVM Classifier: {accuracy_score(y_test, preds)*100}\
     precision_score on test data by SVM Classifier: {precision_score(y_test, preds,average="weighted")*100}\
     recall_score on test data by SVM Classifier: {recall_score(y_test, preds,average="weighted")*100}\
     f1_score on test data by SVM Classifier: {f1_score(y_test, preds,average="weighted")*100}"
)
# cf_matrix = confusion_matrix(y_test, preds)
# plt.figure(figsize=(12, 8))
# sns.heatmap(cf_matrix, annot=True)
# plt.title("Confusion Matrix for SVM Classifier on Test Data")
# plt.show()

# Training and testing Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
preds = nb_model.predict(X_test)
print(
    f"Accuracy on train data by GaussianNB Classifier: {accuracy_score(y_train, nb_model.predict(X_train))*100}\
     precision_score on train data by GaussianNB Classifier: {precision_score(y_train, nb_model.predict(X_train),average="weighted")*100}\
     recall_score on train data by GaussianNB Classifier: {recall_score(y_train, nb_model.predict(X_train),average="weighted")*100}\
     f1_score on train data by GaussianNB Classifier: {f1_score(y_train, nb_model.predict(X_train),average="weighted")*100}"
)

print(
    f"Accuracy on test data by Naive Bayes Classifier: {accuracy_score(y_test, preds)*100}\
     precision_score on test data by Naive Bayes Classifier: {precision_score(y_test, preds,average="weighted")*100}\
     recall_score on test data by Naive Bayes Classifier: {recall_score(y_test, preds,average="weighted")*100}\
     f1_score on test data by Naive Bayes Classifier: {f1_score(y_test, preds,average="weighted")*100}"
)
# cf_matrix = confusion_matrix(y_test, preds)
# plt.figure(figsize=(12, 8))
# sns.heatmap(cf_matrix, annot=True)
# plt.title("Confusion Matrix for Naive Bayes Classifier on Test Data")
# plt.show()

# Training and testing Random Forest Classifier
rf_model = RandomForestClassifier(random_state=18)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
print(
    f"Accuracy on train data by Random Forest Classifier: {accuracy_score(y_train, rf_model.predict(X_train))*100}\
     precision_score on train data by Random Forest Classifier: {precision_score(y_train, rf_model.predict(X_train),average="weighted")*100}\
     recall_score on train data by Random Forest Classifier: {recall_score(y_train, rf_model.predict(X_train),average="weighted")*100}\
     f1_score on train data by Random Forest Classifier: {f1_score(y_train, rf_model.predict(X_train),average="weighted")*100}"
)

print(
    f"Accuracy on test data by Random Forest Classifier: {accuracy_score(y_test, preds)*100}\
     precision_score on test data by Random Forest Classifier: {precision_score(y_test, preds,average="weighted")*100}\
     recall_score on test data by Random Forest Classifier: {recall_score(y_test, preds,average="weighted")*100}\
     f1_score on test data by Random Forest Classifier: {f1_score(y_test, preds,average="weighted")*100}"
)

# Training and testing Random Forest Classifier
dt_model = DecisionTreeClassifier(criterion="gini")
dt_model.fit(X_train, y_train)
preds = dt_model.predict(X_test)
print(
    f"Accuracy on train data by Random Forest Classifier: {accuracy_score(y_train, dt_model.predict(X_train))*100}\
     precision_score on train data by Random Forest Classifier: {precision_score(y_train, dt_model.predict(X_train),average="weighted")*100}\
     recall_score on train data by Random Forest Classifier: {recall_score(y_train, dt_model.predict(X_train),average="weighted")*100}\
     f1_score on train data by Random Forest Classifier: {f1_score(y_train, dt_model.predict(X_train),average="weighted")*100}"
)

print(
    f"Accuracy on test data by Random Forest Classifier: {accuracy_score(y_test, preds)*100}\
     precision_score on test data by Random Forest Classifier: {precision_score(y_test, preds,average="weighted")*100}\
     recall_score on test data by Random Forest Classifier: {recall_score(y_test, preds,average="weighted")*100}\
     f1_score on test data by Random Forest Classifier: {f1_score(y_test, preds,average="weighted")*100}"
)

# cf_matrix = confusion_matrix(y_test, preds)
# plt.figure(figsize=(12, 8))
# sns.heatmap(cf_matrix, annot=True)
# plt.title("Confusion Matrix for Random Forest Classifier on Test Data")
# plt.show()

In [ ]:
# test data preprocessing
test_data = pd.read_csv("data/symptoms_Data_Testing.csv")
test_data = data_processing(test_data)
test_X = test_data.iloc[:, :-1]
test_Y = encoder.transform(test_data.iloc[:, -1])

In [694]:
test_X = selector.fit_transform(test_X, test_Y)
selected_features = selector.get_support(indices=True)

In [ ]:
# Training the models on whole data
final_svm_model = SVC()
final_nb_model = GaussianNB()
final_rf_model = RandomForestClassifier(random_state=18)
final_dt_model = DecisionTreeClassifier(criterion="gini")
final_svm_model.fit(X, y)
final_nb_model.fit(X, y)
final_rf_model.fit(X, y)
final_dt_model.fit(X, y)

# Making prediction by take mode of predictions
# made by all the classifiers
svm_preds = final_svm_model.predict(test_X)
nb_preds = final_nb_model.predict(test_X)
rf_preds = final_rf_model.predict(test_X)
dt_preds = final_dt_model.predict(test_X)


from scipy import stats

final_preds = [
    stats.mode([i, j, k])[0] for i, j, k in zip(svm_preds, dt_preds, rf_preds)
]

print(
    f"Accuracy on Test dataset by the combined model: {accuracy_score(test_Y, final_preds)*100}\
     precision_score on Test dataset by the combined model: {precision_score(test_Y, final_preds,average="weighted")*100}\
     recall_score on Test dataset by the combined model: {recall_score(test_Y, final_preds,average="weighted")*100}\
     f1_score on Test dataset by the combined model: {f1_score(test_Y, final_preds,average="weighted")*100}"
)

# cf_matrix = confusion_matrix(test_Y, final_preds)
# plt.figure(figsize=(12, 8))

# sns.heatmap(cf_matrix, annot=True)
# plt.title("Confusion Matrix for Combined Model on Test Dataset")
# plt.show()

In [ ]:
symptoms = selected_columns


import random

# Randomly select 3 values
if len(selected_columns) >= 3:
    random_values = random.sample(selected_columns, 3)
    print("Randomly selected columns:", random_values)
else:
    print("The list has fewer than 3 elements.")

# Creating a symptom index dictionary to encode the
# input symptoms into numerical form
symptom_index = {}
for index, value in enumerate(symptoms):
    symptom_index[value] = index

data_dict = {"symptom_index": symptom_index, "predictions_classes": encoder.classes_}


# Defining the Function
# Input: string containing symptoms separated by commas
# Output: Generated predictions by models
def predictDisease(symptoms):
    # creating input data for the models
    input_data = [0] * len(data_dict["symptom_index"])
    for symptom in symptoms:
        index = data_dict["symptom_index"][symptom]
        input_data[index] = 1

    # reshaping the input data and converting it
    # into suitable format for model predictions
    input_data = np.array(input_data).reshape(1, -1)

    # generating individual outputs
    rf_prediction = data_dict["predictions_classes"][
        final_rf_model.predict(input_data)[0]
    ]
    svm_prediction = data_dict["predictions_classes"][
        final_svm_model.predict(input_data)[0]
    ]
    dt_prediction = data_dict["predictions_classes"][
        final_dt_model.predict(input_data)[0]
    ]

    # making final prediction by taking mode of all predictions
    # Use statistics.mode instead of scipy.stats.mode
    import statistics

    final_prediction = statistics.mode([rf_prediction, svm_prediction])
    predictions = {
        "rf_model_prediction": rf_prediction,
        "svm_model_prediction": svm_prediction,
        "dt_model_prediction": dt_prediction,
        "final_prediction": final_prediction,
    }
    return predictions


# Testing the function
print(
    predictDisease(random_values)
)